In [ ]:
%pip install transformers

In [ ]:
!git clone https://github.com/huggingface/transformers.git
%cd transformers
%pip install -e 

In [ ]:
pip install transformers -U

In [1]:
import transformers

In [10]:
from datasets import load_dataset
import torch
import transformers

# Load the dataset
data = load_dataset("ruslanmv/ai-medical-chatbot")


#Dataset Size

In [11]:
# Check the size of the dataset
dataset_size = len(data)
print("Dataset size:", dataset_size)
# Print information about the dataset
print(data)

# Specify the split (e.g., "train")
split = "train"

# Print the first sample in the specified split
print(data[split][0])


Dataset size: 1
DatasetDict({
    train: Dataset({
        features: ['Description', 'Patient', 'Doctor'],
        num_rows: 256916
    })
})
{'Description': 'Q. What does abutment of the nerve root mean?', 'Patient': 'Hi doctor,I am just wondering what is abutting and abutment of the nerve root means in a back issue. Please explain. What treatment is required for\xa0annular bulging and tear?', 'Doctor': 'Hi. I have gone through your query with diligence and would like you to know that I am here to help you. For further information consult a neurologist online -->'}


In [12]:
# Specify the split for training
train_data = data["train"]

In [ ]:
print(train_data)

In [14]:
# Define a tokenizer
tokenizer = transformers.GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token

# Define a function to preprocess the data
def preprocess_data(example):
    example['tokenized_description'] = tokenizer(example['Description'], truncation=True, padding='max_length', max_length=128)
    example['tokenized_doctor'] = tokenizer(example['Doctor'], truncation=True, padding='max_length', max_length=128)
    return example

# Preprocess the training data
train_data = train_data.map(preprocess_data)

# Define the model
model = transformers.GPT2LMHeadModel.from_pretrained("gpt2")

# Define the training arguments
training_args = transformers.TrainingArguments(
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    warmup_steps=2,
    max_steps=10,
    learning_rate=2e-4,
    fp16=False,  # Disable FP16 training
    logging_steps=1,
    output_dir="outputs",
    optim="paged_adamw_8bit"
)

# Define the data collator
data_collator = transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False)

# Define the trainer
trainer = transformers.Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_data  # Use the preprocessed training data
)
# # Train the model
# trainer.train()

# # Save the model
# model.save_pretrained("outputs")

In [ ]:
len(trainer)

In [15]:
# # Train the model
# trainer.train()

# # Save the model
# model.save_pretrained("outputs")

# # Load the trained model
# model = transformers.GPT2LMHeadModel.from_pretrained("outputs")

# # Set the model to evaluation mode
# model.eval()
# Train the model
trainer.train()

# Save the model
model.save_pretrained("outputs")

# # Load the trained model
# model = transformers.GPT2LMHeadModel.from_pretrained("outputs")

# # Set the model to evaluation mode
# model.eval()



  0%|          | 0/10 [00:00<?, ?it/s]

IndexError: Invalid key: 86466 is out of bounds for size 0

In [ ]:
# Define the input text
text = "My blood sugar keep rising too much every time I eat sweet do i have any problem?"

# Tokenize the input text
input_ids = tokenizer.encode(text, return_tensors="pt")

# Generate the doctor's response
output_ids = model.generate(input_ids.to(device), max_length=128, num_return_sequences=1)

# Decode and print the response
output_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)
print(output_text)